In [1]:
import plotly.plotly as py
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import HoverTool, NumeralTickFormatter
from bokeh.layouts import gridplot
from bokeh.charts import Bar,output_file, Scatter, BoxPlot
# Import necessary module
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# Create engine: engine
engine = create_engine('sqlite:///programming.sqlite')

In [3]:
# What are the tables in the database?

# Save the table names to a list: table_names
table_names  = engine.table_names()

# Print the table names to the shell
print(table_names)

# See the data model here: http://chinookdatabase.codeplex.com/wikipage?title=Chinook_Schema&referringTitle=Home


['Grade']


In [4]:
# The power of SQL lies in relationships between tables: INNER JOIN

# Execute query and store records in DataFrame: df
data = pd.read_sql_query("select * from Grade",engine)

# drop the unnecessary column
data = data.drop('index', 1)

# print the number of unique students
print(len(data['Aluno_ID'].unique()))

# Print head of DataFrame
data.head()


766


,Aluno_ID,Período,Disciplina,Situaçăo,Média Final,Unidade I,Unidade II,Unidade III
0,0,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.0,4.9,9.0,7.0
1,1,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.3,8.0,7.0,7.0
2,2,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,9.3,9.5,8.3,10.0
3,3,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.3,6.5,7.0,8.3
4,4,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.5,5.5,8.0,9.0


In [5]:
df = pd.read_sql_query("SELECT distinct Disciplina from Grade", engine)

df

,Disciplina
0,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO
1,IMD0012.1 - PRÁTICAS DE TÉCNICAS DE PROGRAMAÇĂO
2,IMD0029 - ESTRUTURA DE DADOS BÁSICAS I
3,IMD0030 - LINGUAGEM DE PROGRAMAÇĂO I
4,IMD0039 - ESTRUTURAS DE DADOS BÁSICAS II
5,IMD0040 - LINGUAGEM DE PROGRAMAÇĂO II


In [6]:
edb1_notas = {}
edb2_notas = {}

def retrieve_edb2_info(aid):
    tmp2 = pd.read_sql_query('''
            SELECT Aluno_ID, [Média Final]
            FROM Grade
            WHERE Aluno_ID = ''' + str(aid) + '''
                AND Disciplina = "IMD0039 - ESTRUTURAS DE DADOS BÁSICAS II"
            LIMIT 1;''', engine
    )
    if tmp2.empty != True:
        try:
            media_edb2 = float(tmp2['Média Final'])
            return media_edb2
        except TypeError:
            return 0.0
    return None

# Execute query and store records in DataFrame: df
data_edb1_edb2 = pd.read_sql_query('''SELECT Aluno_ID, [Média Final]
                       FROM Grade
                       WHERE [Média Final] >= 7.0
                           AND Disciplina = "IMD0029 - ESTRUTURA DE DADOS BÁSICAS I"''',
                       engine)

# print the number of unique students
#print(len(df['Aluno_ID'].unique()))
for aid, med in zip(data_edb1_edb2['Aluno_ID'], data_edb1_edb2['Média Final']):
    edb2 = retrieve_edb2_info(aid)
    if edb2 != None:
        edb1_notas[int(aid)] = float(med)
        edb2_notas[int(aid)] = float(edb2)

In [7]:
red_edb1 = []
red_edb2 = []
green_edb1 = []
green_edb2 = []
blue_edb1 = []
blue_edb2 = []

for x in sorted(edb1_notas.keys()):
    if edb2_notas[int(x)] < 5.0:
        red_edb1.append(edb1_notas[x])
        red_edb2.append(edb2_notas[int(x)])
    elif edb2_notas[int(x)] >= 5.0 and edb2_notas[int(x)] < 7.0:
        green_edb1.append(edb1_notas[x])
        green_edb2.append(edb2_notas[int(x)])
    elif edb2_notas[int(x)] >= 7.0:
        blue_edb1.append(edb1_notas[x])
        blue_edb2.append(edb2_notas[int(x)])

## Análise 4 - Situação dos alunos que foram bem em EDBI

Nesta seção utilizamos a média em EDB2 dos alunos que foram aprovados em EDBI com média $\geq 7,0$, para avaliar se os alunos que foram bem em EDBI também foram bem em EBD2. No gráfico apresentado, o eixo X representa as médias em EDB2 (variando assim de 0 à 10) e o eixo Y representa as médias em EDBI (variando assim de 7 à 10). Sendo assim, um ponto neste gráfico é um par ordenado $(média\_edb1, média\_itp)$ que refere-se à um aluno que cursou EDBI, foi aprovado com média $\geq 7.0$ e pagou a disciplina de EDB2 no semestre seguinte. Esta análise é apresentada de duas formas (através de gráfico de dispersão e gráfico de setores) e é útil para avaliar se os alunos com bom desempenho na disciplina anterior continuavam a apresentar bom desempenho na disciplina seguinte. Nos dois gráficos os alunos estão agrupados de acordo com as cores vermelho ($média\_edb1 \lt 5,0$), verde ($5,0 \leq média\_edb1 \lt 7,0$) e azul ($média\_edb1 \gt 7,0$).

O gráfico de dispersão nos mostra em detalhes o desempenho em EDB2 daqueles que foram aprovados em EDBI. A partir da dispersão dos pontos percebemos que os alunos que foram aprovados com $média \geq 7,0$ em EDB1 tiveram um desempenho muito bom em EDB2, com um total de aproximadamente $50\%$ da turma.

Podemos ver também, que o número de reprovados corresponde a apenas $12.9\%$ do total de alunos, e cerca de $37\%$ dos alunos com média $5,0 \leq média\_edb1 \lt 7,0$, mostrando que $87\%$ dos que vão bem em EDB1, conseguem ser aprovados em EDB2.

O gráfico de setores resume esses resultados de cada uma das categorias.

A partir disso concluímos que é importante e útil aproveitar a disciplina de EDB1 para conseguir aproveitar a disciplina seguinte, que é EDB2.

In [8]:
p = figure(plot_width = 900, plot_height = 500, x_axis_label = 'Média em EDB2', y_axis_label = 'Média em EDB I',
           title = 'Desempenho dos alunos que cursaram EDB1 e EDB2 em seguida')

p.circle(x = red_edb2, y = red_edb1, color = "red", size = 7)
p.circle(x = green_edb2, y = green_edb1, color = "green", size = 7)
p.circle(x = blue_edb2, y = blue_edb1, color = "blue", size = 7)

print('Vermelho: ' + str(len(red_edb2)))
print('Verde: ' + str(len(green_edb2)))
print('Azul: ' + str(len(blue_edb2)))

# Load data from dataframes into figure object
labels = 'Média < 5.0', '5.0 <= Média < 7.0', 'Média >= 7.0'
sizes = [len(red_edb2), len(green_edb2), len(blue_edb2)]
fig = {
    'data': [
        {
            'labels': labels,
            'values': sizes,
            'marker': {'colors': ['red', 'green', 'blue']},
            'type': 'pie',
            'name': 'notas',
            'domain': {'x': [.20, 1]},
            'hoverinfo':'label+percent+name+value',
            'textposition': 'outside',
            'hole': '.4'
        },
        
    ],
    'layout':{
        'title': 'Média Final em EDB II dos alunos que tiveram média final em EDB I acima de 7',
        'annotations':[
                {
                    'font': {'size': 20},
                    'showarrow': False,
                    'text': 'Situação',
                    'x': 0.6,
                    'y': 0.5
                }
        ]
    }
}


output_notebook()
show(p)

Vermelho: 11
Verde: 32
Azul: 42


Loading BokehJS ...

In [9]:
# Plot pie charts
py.iplot(fig)

In [10]:
data_rep = pd.read_sql_query('''SELECT Aluno_ID, Período, Disciplina
                       FROM Grade
                       WHERE Situaçăo = "REPROVADO"''',
                       engine)
data_rep.head()

,Aluno_ID,Período,Disciplina
0,28,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO
1,29,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO
2,30,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO
3,31,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO
4,32,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO


In [11]:
data_rep['count'] = 0
data_rep = pd.DataFrame(data_rep.groupby(['Disciplina','Período'])['count'].count()).reset_index()
data_rep.head()

,Disciplina,Período,count
0,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,2014.1,13
1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,2014.2,18
2,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,2015.1,6
3,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,2015.2,29
4,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,2016.1,20


## Análise 1 - Número de Reprovados x Disciplina em todos os anos.

Nesta seção utilizamos os dados do número de reprovados em cada uma das disciplinas disponíveis, em todos os anos.

Como podemos ver abaixo,as disciplinas Introdução às Técnicas de Programação (ITP) e Práticas de Técnicas de Programação(PTP), Estruturas de dados Básicas I (EDBI) e Laboratório de Programação I(LPI), talvez por serem disciplinas iniciais e primeiros contatos dos alunos com programação, possuem um número de reprovações bem elevado em relação a EDBII e LP2, acumulando aproximadamente 100 reprovações no total.

In [12]:
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,wheel_zoom, pan'
Bar_Reprovados = Bar(data_rep, 'Disciplina', values='count',color = 'Disciplina', xlabel="Disciplina",  
         ylabel = "Nº de Reprovados", title="Disciplinas pelo Nº de Reprovados", 
         legend = False,tools=[TOOLS])

output_notebook()
show(Bar_Reprovados)

Loading BokehJS ...

In [13]:
data_2 = pd.read_sql_query("select * from Grade",engine)
dataYear2014_nota1 = data_2[data_2["Período"] == 2014.1]
dataYear2015_nota1 = data_2[data_2["Período"] == 2015.1]
dataYear2016_nota1 = data_2[data_2["Período"] == 2016.1]
dataYear2014_nota2 = data_2[data_2["Período"] == 2014.2]
dataYear2015_nota2 = data_2[data_2["Período"] == 2015.2]
dataYear2016_nota2 = data_2[data_2["Período"] == 2016.2]


NotasYear2014_1 = BoxPlot(dataYear2014_nota1, values='Média Final', label='Disciplina', color = 'Disciplina', legend = False, 
             title = "BoxPlot das disciplinas pelas notas, Ano 2014.1", tools = [TOOLS])

NotasYear2015_1 = BoxPlot(dataYear2015_nota1, values='Média Final', label='Disciplina', color = 'Disciplina',legend = False,
            title = "BoxPlot das disciplinas pelas notas, Ano 2015.1", tools = [TOOLS])

NotasYear2016_1 = BoxPlot(dataYear2016_nota1, values='Média Final', label='Disciplina', color = 'Disciplina',legend = False,
            title = "BoxPlot das disciplinas pelas notas, Ano 2016.1", tools = [TOOLS])

NotasYear2014_2 = BoxPlot(dataYear2014_nota2, values='Média Final', label='Disciplina', color = 'Disciplina', legend = False, 
             title = "BoxPlot das disciplinas pelas notas, Ano 2014", tools = [TOOLS])

NotasYear2015_2 = BoxPlot(dataYear2015_nota2, values='Média Final', label='Disciplina', color = 'Disciplina',legend = False,
            title = "BoxPlot das disciplinas pelas notas, Ano 2015.1", tools = [TOOLS])

NotasYear2016_2 = BoxPlot(dataYear2016_nota2, values='Média Final', label='Disciplina', color = 'Disciplina',legend = False,
            title = "BoxPlot das disciplinas pelas notas, Ano 2016.1", tools = [TOOLS])

## Análise 2 - Disciplinas x Notas em cada ano.

Nesta seção utilizamos os dados de cada uma das disciplinas pelas notas, em cada ano em que foram ofertadas.

Analisando as disciplinas, a grande maioria delas se mantém no mesmo nível ao longo dos períodos. Um destaque para essa análise é ofato de que as medianas estão sempre próximas de 5 e 6, o que podemos concluir que em algumas disciplinas, como EDBI e LPI, a maioria dos alunos ou está sendo reprovada ou passando com dificudade. Também podemos notar a disciplina de LPI que no ano de 2016 teve sua mediana menor que os anos anteriores, na qual no ano de 2016, cerca de $50\%$ dos alunos ficaram com média bem próximos a 5 e mediana da turma aproximadamente 3,2.

In [14]:
row1 = [NotasYear2014_1,NotasYear2014_2]

# Create a list containing plots p3 and p4: row2
row2 = [NotasYear2015_1,NotasYear2015_2]

row3 = [NotasYear2016_1,NotasYear2016_2]

# Create a gridplot using row1 and row2: layout
layout = gridplot([row1,row2,row3],plot_width=480, plot_height=700)

output_notebook()


show(layout)

Loading BokehJS ...

## Análise 3 - Disciplinas x Notas em todos os anos.

Nesta seção utilizamos os dados de cada uma das disciplinas pelas notas, em cada ano em que foram ofertadas.

Analisando as disciplinas, como dito anteriormente, no geral, as notas estão com medianas entre 5 e 6. As disciplinas iniciais ITP, PTP, EDBI e LP1, possuem uma distribuição menos concentra em relação a EDII e LPII. Destque LP2 com mais de $50\%$ da turma acima de 5.2 e com as melhores notas, levando em consideração as disciplinas de programação.

In [15]:
BoxPlot_Nota = BoxPlot(data_2, values='Média Final', label='Disciplina', legend = False, color = 'Disciplina', tools = [TOOLS])

output_notebook()


show(BoxPlot_Nota)

Loading BokehJS ...